# How to start

Before starting you must:
- Have conda installed
- `conda env create -f ess-notebooks-latest.yml python=3.7` . The yaml environment file is part of this repository.
- fetch the data `git clone git@github.com:scipp/ess-notebooks-data.git` somewhere local 
- Generate the `dataconfig.py` file using `make_config.py` located in same directory as this notebook. In general, you simply need to point `make_config.py` to the root directory of data you cloned above. Refer to the help `make_config.py --help` for more information. 



# Experimental Summary

[TODO]


# Script setup

In [ ]:
try:
    import scipp
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e

try:
    import mantid
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e

try:
    import dataconfig
except ImportError as e:
    print(
        "dataconfig is not available. Make sure you have generated it with `make_config.py`."
    )
    raise e

## Set input and output dirs

If your input directory has a different structure this is the cell to modify. 
Additionally the output directory can be renamed too.

In [ ]:
# Lets get everything set up
import os
import sys

import scipp as sc
import numpy as np

import ess.v20.imaging as imaging
import ess.v20.imaging.operations as operations
import dataconfig
from scipy import ndimage, signal

local_data_path = os.path.join('ess', 'v20', 'imaging', 'mpl-metals-experiments')
data_dir = os.path.join(dataconfig.data_root, local_data_path)
tofs_path = os.path.join(data_dir, 'PSI_WFM_samples000_Spectra.txt')

if not os.path.exists(data_dir):
    raise FileNotFoundError("The following data directory does not exist,"
                            f" check your make_config.py:\n{data_dir}")

# Reduction

## Load and scale

In [ ]:
imaging.__file__

In [ ]:
ds = sc.Dataset()
ds.coords["t"] = sc.Variable(["t"],
                             unit=sc.units.us,
                             values=imaging.read_x_values(tofs_path))
ds.coords["t"] *= 1e3


def load_and_scale(data_dir, folder_name, scale_factor):
    to_load = os.path.join(data_dir, folder_name)
    variable = imaging.helper_funcs.fits_to_variable(to_load)
    variable *= scale_factor
    return variable

pulse_number_reference = 1 # TODO - how many pulses counted for? 
pulse_number_sample = 1 # TODO - how many pulses counted for?

ds["reference"] = load_and_scale(data_dir, folder_name="04_OB_6gaps_summed",
                                 scale_factor=pulse_number_reference)
ds["sample"] = load_and_scale(data_dir, folder_name="05_samples_6gaps_summed",
                              scale_factor=pulse_number_sample)

## Add Spectra Coords

In [ ]:
# Adds a coordinate for the spectrum
ds.coords["spectrum"] = sc.Variable(["spectrum"],
                                    values=np.arange(ds["sample"].shape[1]))